In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget
#%matplotlib ipympl

#%reload_ext tensorboard
#%matplotlib qt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import napari

from pathlib import Path
from torchinfo import summary
#from torch_geometric.nn import summary
from skimage.segmentation import slic
from skimage import segmentation
from skimage.exposure import rescale_intensity
from torch_geometric.transforms import ToSLIC
import torch
import SimpleITK as sitk
from hnc_project.pytorch.run_model_torch import RunModel

patch_dir = Path('../../data/HNSCC/HNSCC_Nii_222_50_50_60_Crop_v2')
nii_dir = Path('../../data/HNSCC/HNSCC_Nii_v3')
twomm_dir = Path('../../data/HNSCC/HNSCC_Nii_resample_222')
test_patient = 'HNSCC-01-0233'

In [3]:
image = sitk.ReadImage(nii_dir.joinpath(f"{test_patient}/image.nii.gz"))
struct_gtvp = sitk.ReadImage(nii_dir.joinpath(f"{test_patient}/Struct_GTVp.nii.gz"))
struct_gtvn1 = sitk.ReadImage(nii_dir.joinpath(f"{test_patient}/Struct_GTVn1.nii.gz"))
struct_gtvn2 = sitk.ReadImage(nii_dir.joinpath(f"{test_patient}/Struct_GTVn2.nii.gz"))
image = sitk.Clamp(image, sitk.sitkFloat32, -500, 500)

In [4]:
image_arr = sitk.GetArrayFromImage(image)
struct_gtvp_arr = sitk.GetArrayFromImage(struct_gtvp)
struct_gtvn1_arr = sitk.GetArrayFromImage(struct_gtvn1)
struct_gtvn2_arr = sitk.GetArrayFromImage(struct_gtvn2)
total_struct = struct_gtvp_arr + struct_gtvn1_arr + struct_gtvn2_arr
opp_struct = np.logical_not(total_struct)

In [5]:
image_arr = rescale_intensity(image_arr, in_range=(-500, 500), out_range=(0,1))

In [13]:
slic_gtvp_seg = slic(image_arr, mask=struct_gtvp_arr, compactness=0.3, n_segments=20, max_num_iter=10, channel_axis=None, slic_zero=True)
slic_gtvn1_seg = slic(image_arr, mask=struct_gtvn1_arr, compactness=0.3, n_segments=20, max_num_iter=10, channel_axis=None, slic_zero=True)
slic_gtvn2_seg = slic(image_arr, mask=struct_gtvn2_arr, compactness=0.3, n_segments=20, max_num_iter=10, channel_axis=None, slic_zero=True)
#slic_seg = slic(image_arr, mask=struct_arr, n_segments=50, max_num_iter=10, channel_axis=None, slic_zero=True)

In [57]:
slic_seg = slic(image_arr, compactness=0.3, n_segments=2000, max_num_iter=10, sigma=0.5, channel_axis=None, slic_zero=False)
#slic_seg = slic(image_arr, mask=opp_struct, compactness=0.01, n_segments=1500, max_num_iter=10, channel_axis=None, slic_zero=True)

In [58]:
boundaries = []
boundaries_gtvp = []
boundaries_gtvn1 = []
boundaries_gtvn2 = []
for idx, slice in enumerate(image_arr):
    #if idx != 140: continue
    #slic_slices.append(slic(image_arr[idx], compactness=0.3, n_segments=400, max_num_iter=10, channel_axis=None, slic_zero=True))
    #boundaries.append(segmentation.mark_boundaries(slice, slic_slices[idx], color=(1,1,1)))
    boundaries.append(segmentation.mark_boundaries(slice, slic_seg[idx], color=(1,1,0)))
    #boundaries_gtvp.append(segmentation.mark_boundaries(slice, slic_gtvp_seg[idx], color=(1,0,0)))
    #boundaries_gtvn1.append(segmentation.mark_boundaries(slice, slic_gtvn1_seg[idx], color=(0,1,1)))
    #boundaries_gtvn2.append(segmentation.mark_boundaries(slice, slic_gtvn2_seg[idx], color=(0,1,1)))
    #boundaries.append(segmentation.find_boundaries(slic_seg[idx], mode='outer'))
    #boundaries_gtvp.append(segmentation.find_boundaries(slic_gtvp_seg[idx], mode='outer'))
    #boundaries_gtvn1.append(segmentation.find_boundaries(slic_gtvn1_seg[idx], mode='outer'))
    #boundaries_gtvn2.append(segmentation.find_boundaries(slic_gtvn2_seg[idx], mode='outer'))
boundaries = np.array(boundaries)
#boundaries_gtvp = np.array(boundaries_gtvp)
#boundaries_gtvn1 = np.array(boundaries_gtvn1)
#boundaries_gtvn2 = np.array(boundaries_gtvn2)

In [59]:
#viewer = napari.view_image(np.where(np.logical_not(boundaries), image_arr, 1000.)
viewer = napari.view_image(boundaries)
#viewer.add_image(boundaries_gtvp)
#viewer.add_image(boundaries_gtvn1)
#viewer.add_image(boundaries_gtvn2)

D:\work\Miniconda3\envs\pytorch_gpu_v2\Lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


In [13]:
slic_transform = ToSLIC(add_seg=True, add_img=True, slic_zero=True)